# Compare Relationships across all labels between 2 versions

## Importing

In [5]:
import os
import logging
import configparser
import pandas as pd
from neo4j import GraphDatabase


## Get Current Path

In [6]:
# Setup
current_path = os.path.dirname(os.path.realpath(__file__))
# current_path = os.path.dirname(os.path.realpath('f0153.ipynb'))

## Read Config File 

In [7]:
config = configparser.ConfigParser()
config.read(f"{current_path}/config.ini")

neo4j_uri_old = config['DEFAULT']['Neo4j-Uri-old']
neo4j_uri_new = config['DEFAULT']['Neo4j-Uri-new']
username = config['DEFAULT']['Neo4j-Username']
password_old = config['DEFAULT']['Neo4j-Password-old']
password_new = config['DEFAULT']['Neo4j-Password-new']
output_folder = config['DEFAULT']['Output-Folder']

## Setup Log 

In [8]:
logging.basicConfig(
    filename=f"{current_path}/relationship_diff_all.log",
    level=logging.INFO,
    format='%(asctime)s | %(levelname)s | %(message)s'
)

print("Comparing relationship differences by start label")

Comparing relationship differences by start label


## Helper Functions to Query Relationships 

In [9]:
def run_query(driver, query):
    with driver.session() as session:
        result = session.run(query)
        return pd.DataFrame([dict(r) for r in result])

def get_graph_version(driver):
    query = "MATCH (n:version) RETURN n.version AS version LIMIT 1"
    try:
        df = run_query(driver, query)
        return df.iloc[0]['version']
    except Exception as e:
        logging.warning(f"Couldn't fetch version info: {e}")
        return "unknown"

def get_relationships_by_label(driver, label):
    query = f"""
    MATCH (a:`{label}`)-[r]->(b)
    RETURN 
        '{label}' AS start_node_type,
        a.source AS start_node_source,
        type(r) AS relationship_type,
        labels(b) AS target_node_labels,
        b.source AS target_node_source,
        count(*) AS count
    """
    return run_query(driver, query)

core_labels = ["researcher", "publication", "dataset", "grant", "organisation"]

def resolve_target_node_type(df):
    def match_label(label_list):
        for lbl in core_labels:
            if lbl in label_list:
                return lbl
        return "unknown"
    
    df["target_node_type"] = df["target_node_labels"].apply(match_label)
    return df.drop(columns=["target_node_labels"])

## Compare Relationships for All Labels

In [10]:
# Labels to compare
labels = core_labels
all_diffs = []

# Connect to both graphs
driver_old = GraphDatabase.driver(neo4j_uri_old, auth=(username, password_old))
driver_new = GraphDatabase.driver(neo4j_uri_new, auth=(username, password_new))

# Get version names dynamically
version_old = get_graph_version(driver_old)
version_new = get_graph_version(driver_new)

for label in labels:
    try:
        df_old = get_relationships_by_label(driver_old, label)
        df_old = resolve_target_node_type(df_old)
        df_new = get_relationships_by_label(driver_new, label)
        df_new = resolve_target_node_type(df_new)

        merge_cols = [
            "start_node_type", "start_node_source",
            "relationship_type", "target_node_type", "target_node_source"
        ]

        merged_df = pd.merge(
            df_old, df_new,
            on=merge_cols,
            how="outer",
            suffixes=(f"_{version_old}", f"_{version_new}")
        )

        merged_df.fillna(0, inplace=True)

        merged_df["diff"] = (
            merged_df[f"count_{version_new}"] - merged_df[f"count_{version_old}"]
        )
        ordered_cols = [
        "start_node_type", "start_node_source", "relationship_type",
        "target_node_type", "target_node_source",
        f"count_{version_old}", f"count_{version_new}", "diff"
        ]
        merged_df = merged_df[ordered_cols]
        all_diffs.append(merged_df)

        
        logging.info(f"✅ Compared relationships for: {label}")

    except Exception as e:
        logging.error(f"❌ Failed comparing relationships for {label}: {e}")

driver_old.close()
driver_new.close()

if all_diffs:
    final_relationship_diff = pd.concat(all_diffs)
    final_relationship_diff.sort_values(
        by=["start_node_type", "relationship_type", "diff"], ascending=[True, True, False], inplace=True
    )
    final_relationship_diff.reset_index(drop=True, inplace=True)
    final_relationship_diff
else:
    print("⚠️ No relationship differences to report.")
    final_relationship_diff = pd.DataFrame()
final_relationship_diff

,start_node_type,start_node_source,relationship_type,target_node_type,target_node_source,count_1.0.3,count_1.1.11,diff
0,dataset,orcid.org,orcid,dataset,datacite.org,2341.0,14575.0,12234.0
1,dataset,orcid.org,orcid,publication,crossref.org,252.0,971.0,719.0
2,dataset,orcid.org,orcid,dataset,crossref.org,1314.0,1459.0,145.0
3,dataset,orcid.org,orcid,unknown,crossref.org,1.0,1.0,0.0
4,dataset,orcid.org,orcid,publication,datacite.org,12153.0,433.0,-11720.0
...,...,...,...,...,...,...,...,...
60,researcher,orcid.org,orcid,dataset,orcid.org,15206.0,15764.0,558.0
61,researcher,orcid.org,orcid,dataset,crossref.org,1594.0,1787.0,193.0
62,researcher,orcid.org,orcid,grant,crossref.org,0.0,1.0,1.0
63,researcher,orcid.org,orcid,publication,datacite.org,29773.0,24611.0,-5162.0


## Save Markdown Report

In [13]:
output_file = f"{output_folder}/neo4jreports/relationship_diff_all_labels.md"

try:
    with open(output_file, "w") as f:
        f.write(final_relationship_diff.to_markdown(index=False))
    logging.info(f"Saved node source diff for all labels to {output_file}")
    print(f"Saved node source diff for all labels to {output_file}")
except Exception as e:
    logging.error(f"Failed to write final Markdown file: {e}")
    print(f"Error writing Markdown file: {e}")

Saved node source diff for all labels to /Users/sriharshithathoram/Desktop/neo4j-is/neo4jreports/relationship_diff_all_labels.md
